In [8]:
import os
import requests
import pandas as pd
from io import StringIO

# ClickHouse Cloud API credentials
key_id = os.environ.get('CH_KEY_ID')
key_secret = os.environ.get('CH_KEY_SECRET')
service_id = '88ac54be-2166-445d-9172-dc3173309069'

def run_clickhouse_query(sql_query, format_type='CSV'):
    """Execute SQL query against ClickHouse Cloud using REST API"""
    url = f'https://queries.clickhouse.cloud/service/{service_id}/run'
    
    response = requests.post(
        url,
        auth=(key_id, key_secret),
        headers={'Content-Type': 'application/json'},
        params={'format': format_type},
        json={'sql': sql_query},
        timeout=30
    )
    
    response.raise_for_status()
    return response.text

# sanity check
version_result = run_clickhouse_query("SELECT version()")
print("ClickHouse version:", version_result.strip())

# example query with results
sample_query = """
SELECT * FROM flights_df
LIMIT 10
"""

csv_result = run_clickhouse_query(sample_query)
sample_df = pd.read_csv(StringIO(csv_result))
print("\nSample query results:")
print(sample_df.head())

ClickHouse version: "25.4.1.37654"

Sample query results:
   AS  ANC  SEA   5  430  -11  -22  1448  4  169  194  205  2015-01-01
0  AA  LAX  PBI  10  750   -8   -9  2330  4  263  279  280  2015-01-01
1  US  SFO  CLT  20  806   -2    5  2296  4  266  293  286  2015-01-01
2  AA  LAX  MIA  20  805   -5   -9  2342  4  258  281  285  2015-01-01
3  AS  SEA  ANC  25  320   -1  -21  1448  4  199  215  235  2015-01-01
4  DL  SFO  MSP  25  602   -5    8  1589  4  206  230  217  2015-01-01


In [ ]:
clickhouse_flights_grammar = r"""
// ====== Punctuation & tokens (thread spaces explicitly) ======
SP: " "
COMMA: ","
SEMI: ";"
STAR: "*"

// ====== Start ======
start: "SELECT" SP select_list SP "FROM" SP table opt_where opt_group_by opt_order_by opt_limit SEMI

// ====== Table (rename IDENTIFIER to your actual table if you like strictness) ======
table: IDENTIFIER   // e.g. 'flights_df'

// ====== Projections ======
select_list: select_item (COMMA SP select_item)*
select_item: projection [SP "AS" SP alias]

projection: column
          | agg_func "(" measure ")"
          | quantile_fn "(" quant_p ")" "(" measure ")"
          | "count" "(" STAR ")"
          | "countIf" "(" condition ")"
          | mph_expr
          | dep_hour_expr
          | arr_hour_expr

agg_func: "avg" | "sum" | "min" | "max"
quantile_fn: "quantileTDigest" | "quantileExact"
quant_p: /0\.[0-9]{1,2}/             // 0.1 .. 0.99

// what counts as a numeric measure
measure: numeric_field | mph_expr

// ====== Columns you expose directly ======
column: "AIRLINE"
      | "ORIGIN_AIRPORT"
      | "DESTINATION_AIRPORT"
      | "DAY_OF_WEEK"
      | "FLIGHT_DATE"

// numeric fields safe for math/agg
numeric_field: "DEPARTURE_DELAY"
             | "ARRIVAL_DELAY"
             | "DISTANCE"
             | "AIR_TIME"
             | "ELAPSED_TIME"
             | "SCHEDULED_TIME"

// ====== Canonical derived expressions ======
mph_expr: "(" "DISTANCE" SP "*" SP "60" SP "/" SP "AIR_TIME" ")"
dep_hour_expr: "intDiv" "(" "SCHEDULED_DEPARTURE" COMMA SP "100" ")"
arr_hour_expr: "intDiv" "(" "SCHEDULED_ARRIVAL" COMMA SP "100" ")"

// ====== WHERE (only AND, no OR, to keep it robust) ======
opt_where: | SP "WHERE" SP boolean_conj
boolean_conj: condition (SP "AND" SP condition)*

condition: comparison
         | date_between
         | hour_between_dep
         | hour_between_arr
         | airline_eq
         | origin_eq
         | destination_eq
         | earlylate_clause     // e.g., abs(ARRIVAL_DELAY) <= 5

comparison: numeric_field SP comp_op SP number
comp_op: "=" | ">" | "<" | ">=" | "<="

date_between: "FLIGHT_DATE" SP "BETWEEN" SP DATE SP "AND" SP DATE

hour_between_dep: dep_hour_expr SP "BETWEEN" SP hour SP "AND" SP hour
hour_between_arr: arr_hour_expr SP "BETWEEN" SP hour SP "AND" SP hour

airline_eq: "AIRLINE" SP "=" SP UPPER2        // e.g., 'AA'
origin_eq: "ORIGIN_AIRPORT" SP "=" SP UPPER3  // e.g., 'SFO'
destination_eq: "DESTINATION_AIRPORT" SP "=" SP UPPER3

earlylate_clause: "abs" "(" "ARRIVAL_DELAY" ")" SP "<=" SP number
                | "abs" "(" "DEPARTURE_DELAY" ")" SP "<=" SP number
                | "ARRIVAL_DELAY" SP ">=" SP number
                | "DEPARTURE_DELAY" SP ">=" SP number

// ====== GROUP BY ======
opt_group_by: | SP "GROUP" SP "BY" SP group_cols
group_cols: group_expr (COMMA SP group_expr)*
group_expr: column | dep_hour_expr | arr_hour_expr

// ====== ORDER BY ======
opt_order_by: | SP "ORDER" SP "BY" SP order_cols
order_cols: order_expr (COMMA SP order_expr)*
order_expr: (agg_func "(" measure ")" 
            | quantile_fn "(" quant_p ")" "(" measure ")"
            | column 
            | dep_hour_expr 
            | arr_hour_expr 
            | mph_expr) [SP order_dir]
order_dir: "ASC" | "DESC"

// ====== LIMIT ======
opt_limit: | SP "LIMIT" SP NUMBER

// ====== Terminals ======
alias: IDENTIFIER
IDENTIFIER: /[A-Za-z_][A-Za-z0-9_]*/
NUMBER: /[0-9]+/
FLOAT: /[0-9]+\.[0-9]+/
number: NUMBER | FLOAT
DATE: /'[0-9]{4}-[0-9]{2}-[0-9]{2}'/
hour: /([01]?[0-9]|2[0-3])/
UPPER2: /'[A-Z0-9]{2}'/
UPPER3: /'[A-Z0-9]{3}'/
"""

In [ ]:
import os
from openai import AsyncAzureOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

client = AsyncAzureOpenAI(
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY")
)

In [13]:
tool = {
  "type": "custom",
  "name": "clickhouse_sql_grammar",
  "description": (
    "Produces **read-only ClickHouse SQL** over the flights_df table. "
    "Allowed constructs: SELECT, FROM, WHERE (AND only), GROUP BY, ORDER BY, LIMIT; "
    "columns limited to the provided schema; derived exprs allowed: "
    "intDiv(SCHEDULED_DEPARTURE,100), intDiv(SCHEDULED_ARRIVAL,100), (DISTANCE*60/AIR_TIME); "
    "aggregates: avg,sum,min,max,count,countIf(cond),quantileTDigest/Exact(p)(field). "
    "YOU MUST THINK THROUGH COMPLIANCE WITH THE GRAMMAR BEFORE EMITTING SQL."
  ),
  "format": {"type": "grammar", "syntax": "lark", "definition": clickhouse_flights_grammar},
}

prompt = (
  "Generate a query to rank airlines by average ARRIVAL_DELAY for January 2015"
)

resp = await client.responses.create(
  model="gpt-5-mini",
  input=prompt,
  text={"format": {"type": "text"}},
  tools=[tool],
  parallel_tool_calls=False,
)

print(resp.output[1].input)

SELECT AIRLINE, avg(ARRIVAL_DELAY) AS AVG_ARRIVAL_DELAY FROM flights_df WHERE FLIGHT_DATE BETWEEN '2015-01-01' AND '2015-01-31' GROUP BY AIRLINE ORDER BY avg(ARRIVAL_DELAY) DESC;


In [16]:
query = resp.output[1].input

csv_result = run_clickhouse_query(query, format_type='JSONEachRow')
sample_df = pd.read_csv(StringIO(csv_result))
print("\nSample query results:")
print(sample_df.head())


Sample query results:
   {"AIRLINE":"F9"  AVG_ARRIVAL_DELAY:18.104554107482794}
0  {"AIRLINE":"MQ"  AVG_ARRIVAL_DELAY:16.748227424749164}
1  {"AIRLINE":"NK"  AVG_ARRIVAL_DELAY:11.253345533569712}
2  {"AIRLINE":"OO"  AVG_ARRIVAL_DELAY:10.559670781893004}
3  {"AIRLINE":"EV"   AVG_ARRIVAL_DELAY:8.222674011016524}
4  {"AIRLINE":"B6"   AVG_ARRIVAL_DELAY:6.959580076770106}
